In [219]:
#importing pandas and numpy library for reading files and data manupulation
# import re i.r regular expression

import pandas as pd
import numpy as np
import re 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # access convergence dint work
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score




In [206]:
#Read the train and test files

train_data = pd.read_csv('C:/Users/punit/Binary Prediction of Smoker Status using Bio-Signals/train.csv')
test_data = pd.read_csv('C:/Users/punit/Binary Prediction of Smoker Status using Bio-Signals/test.csv')

In [207]:
#Processing the Data

In [186]:

train_data.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [187]:
train_data.columns

Index(['id', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
       'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic',
       'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
       'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST',
       'ALT', 'Gtp', 'dental caries', 'smoking'],
      dtype='object')

In [188]:
#standardize the column names

[re.sub(r'\s|\)|\(','_',i.lower()) for i in train_data.columns]

['id',
 'age',
 'height_cm_',
 'weight_kg_',
 'waist_cm_',
 'eyesight_left_',
 'eyesight_right_',
 'hearing_left_',
 'hearing_right_',
 'systolic',
 'relaxation',
 'fasting_blood_sugar',
 'cholesterol',
 'triglyceride',
 'hdl',
 'ldl',
 'hemoglobin',
 'urine_protein',
 'serum_creatinine',
 'ast',
 'alt',
 'gtp',
 'dental_caries',
 'smoking']

In [189]:
# rename the columns

train_data = train_data.rename(
                        
            columns = { 
                       old: re.sub(r'\s|\)|\(','_', old).lower()
                       for old in train_data.columns
                      }
)

In [190]:
# rename the columns

test_data = test_data.rename(
                        
            columns = { 
                       old: re.sub(r'\s|\)|\(','_', old).lower()
                       for old in test_data.columns
                      }
)

In [191]:
train_data.columns

Index(['id', 'age', 'height_cm_', 'weight_kg_', 'waist_cm_', 'eyesight_left_',
       'eyesight_right_', 'hearing_left_', 'hearing_right_', 'systolic',
       'relaxation', 'fasting_blood_sugar', 'cholesterol', 'triglyceride',
       'hdl', 'ldl', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'ast',
       'alt', 'gtp', 'dental_caries', 'smoking'],
      dtype='object')

In [161]:
test_data.columns

Index(['id', 'age', 'height_cm_', 'weight_kg_', 'waist_cm_', 'eyesight_left_',
       'eyesight_right_', 'hearing_left_', 'hearing_right_', 'systolic',
       'relaxation', 'fasting_blood_sugar', 'cholesterol', 'triglyceride',
       'hdl', 'ldl', 'hemoglobin', 'urine_protein', 'serum_creatinine', 'ast',
       'alt', 'gtp', 'dental_caries'],
      dtype='object')

In [131]:
#Understanding column data

train_data.id

0              0
1              1
2              2
3              3
4              4
           ...  
159251    159251
159252    159252
159253    159253
159254    159254
159255    159255
Name: id, Length: 159256, dtype: int64

In [132]:
train_data.describe(include = 'all')

,id,age,height_cm_,weight_kg_,waist_cm_,eyesight_left_,eyesight_right_,hearing_left_,hearing_right_,systolic,...,hdl,ldl,hemoglobin,urine_protein,serum_creatinine,ast,alt,gtp,dental_caries,smoking
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,...,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,79627.500000,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,...,55.852684,114.607682,14.796965,1.074233,0.892764,25.516853,26.550296,36.216004,0.197996,0.437365
std,45973.391572,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,...,13.964141,28.158931,1.431213,0.347856,0.179346,9.464882,17.753070,31.204643,0.398490,0.496063
min,0.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,...,9.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000,0.000000
25%,39813.750000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000
50%,79627.500000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000
75%,119441.250000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000
max,159255.000000,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,...,136.000000,1860.000000,21.000000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000,1.000000


In [133]:
train_data.shape

(159256, 24)

In [134]:
#check for null values.
#columns dont have any null values

train_data.isnull().sum()

id                     0
age                    0
height_cm_             0
weight_kg_             0
waist_cm_              0
eyesight_left_         0
eyesight_right_        0
hearing_left_          0
hearing_right_         0
systolic               0
relaxation             0
fasting_blood_sugar    0
cholesterol            0
triglyceride           0
hdl                    0
ldl                    0
hemoglobin             0
urine_protein          0
serum_creatinine       0
ast                    0
alt                    0
gtp                    0
dental_caries          0
smoking                0
dtype: int64

In [192]:
#drop columns not needed to build the model.
# in this case dropping id

x_train = train_data.drop('id', axis = 1)
x_test = test_data.drop('id', axis = 1) 


In [193]:
#drop

x_train = x_train.drop('smoking', axis = 1)
y_train = train_data['smoking']

x_test = test_data.drop('smoking', axis = 1)



#ignore the error

KeyError: "['smoking'] not found in axis"

In [195]:
# split the data

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,train_size = 0.8, test_size = 0.2 , random_state = 0 )

In [196]:
# create and train simple Decison tree model

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [198]:
# make predictions on training data

y_pred = model.predict(x_val)

In [199]:
# auc score for train data

train_auc = roc_auc_score(y_val,y_pred)
print( "AUC score for train data is : ", train_auc)

AUC score for train data is :  0.6912365536937873


In [208]:
# create and train Random FOrest model

model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_val)

train_auc = roc_auc_score(y_val,y_pred)
print( "AUC score for train data is : ", train_auc)

AUC score for train data is :  0.7785971356107989


In [216]:
# Optimizing the XGBoost MOdel

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [217]:
# Optimizing xgboost model

def objective(space):
    model = xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( x_train, y_train), ( x_val, y_val)]
    
    model.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }



In [220]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                 
0.7674871279668467                                     
  1%|          | 1/100 [00:07<11:35,  7.03s/trial, best loss: -0.7674871279668467]

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.7620871530830089                                                                
  2%|▏         | 2/100 [00:12<10:26,  6.39s/trial, best loss: -0.7674871279668467]

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.7687743312821801                                                                
  3%|▎         | 3/100 [00:22<12:21,  7.65s/trial, best loss: -0.7687743312821801]

...

SCORE:                                                                             
0.7664196910712043                                                                 
 98%|█████████▊| 98/100 [17:03<00:21, 10.88s/trial, best loss: -0.7712859475072209]

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                             
0.7699045585834484                                                                 
 99%|█████████▉| 99/100 [17:14<00:10, 10.86s/trial, best loss: -0.7712859475072209]

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\punit\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                             
0.7701243250031395                                                                 
100%|██████████| 100/100 [17:25<00:00, 10.46s/trial, best loss: -0.7712859475072209]


In [230]:
# best score

# Create and train the XGBoost model
model = xgb.XGBClassifier(n_estimators= 200, learning_rate=0.3, max_depth=5)
model.fit(x_train, y_train)

# Make predictions on training data
y_pred = model.predict(x_val)

# Calculate the AUC score for train data
train_auc = roc_auc_score(y_val, y_pred)

print("AUC score for train data is:", train_auc)


AUC score for train data is: 0.7885149361450872


In [254]:
# optimizing for better auc 
# NEw best score

# Create and train the XGBoost model
model = xgb.XGBClassifier(n_estimators= 500, learning_rate=0.09, max_depth=5)
model.fit(x_train, y_train)

# Make predictions on training data
y_pred = model.predict(x_val)

# Calculate the AUC score for train data
train_auc = roc_auc_score(y_val, y_pred)

print("AUC score for train data is:", train_auc)


AUC score for train data is: 0.7903048261783471


In [212]:

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_val)

train_auc = roc_auc_score(y_val, y_pred)
print("AUC score for train data is:", train_auc)


AUC score for train data is: 0.7811422362785434
